In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv("iris-species/Iris.csv")

In [3]:
data.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
# Apply one-hot encoding
def one_hot(value):
    if value == "Iris-setosa":
        return 0
    elif value == "Iris-versicolor":
        return 1
    else:
        return 2

In [5]:
data['Species'] = data['Species'].apply(one_hot)

In [6]:
data['Species'].unique()

array([0, 1, 2])

In [7]:
# Perform train-test split

In [8]:
X = data.drop(['Species','Id'], axis=1)
y = data['Species']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)

### Classification Using TFEstimator API

In [10]:
data.columns

Index(['Id', 'SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm',
       'Species'],
      dtype='object')

In [11]:
# Create numeric feature columns

SepalLengthCm = tf.feature_column.numeric_column('SepalLengthCm')
SepalWidthCm = tf.feature_column.numeric_column('SepalWidthCm')
PetalLengthCm = tf.feature_column.numeric_column('PetalLengthCm')
PetalWidthCm = tf.feature_column.numeric_column('PetalWidthCm')
Species = tf.feature_column.numeric_column('Species')

feat_cols = [SepalLengthCm, SepalWidthCm, PetalLengthCm, PetalWidthCm]

In [12]:
print(X_train.shape)
print(y_train.shape)

(105, 4)
(105,)


In [26]:
# Create an input function that will be fed into the input dictionary
input_fnc = tf.estimator.inputs.pandas_input_fn(x=X_train, y=y_train, shuffle=True, batch_size=10, num_epochs=50)

In [27]:
# Create deep the model
model = tf.estimator.DNNClassifier(hidden_units=[10,10,10,4],feature_columns=feat_cols, n_classes=3,
                                   optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.003))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_master': '', '_log_step_count_steps': 100, '_keep_checkpoint_max': 5, '_task_id': 0, '_task_type': 'worker', '_session_config': None, '_service': None, '_keep_checkpoint_every_n_hours': 10000, '_num_worker_replicas': 1, '_tf_random_seed': None, '_model_dir': '/var/folders/y0/f6x1qz191gxdq8fj4lpy82l00000gn/T/tmpjj9r6j1j', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1023cda20>, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_save_summary_steps': 100, '_is_chief': True}


In [28]:
# Train the model
model.train(input_fn=input_fnc, steps=500)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/y0/f6x1qz191gxdq8fj4lpy82l00000gn/T/tmpjj9r6j1j/model.ckpt.
INFO:tensorflow:step = 1, loss = 11.1327
INFO:tensorflow:global_step/sec: 362.226
INFO:tensorflow:step = 101, loss = 3.91553 (0.279 sec)
INFO:tensorflow:global_step/sec: 431.624
INFO:tensorflow:step = 201, loss = 2.47501 (0.232 sec)
INFO:tensorflow:global_step/sec: 393.19
INFO:tensorflow:step = 301, loss = 3.33306 (0.255 sec)
INFO:tensorflow:global_step/sec: 414.26
INFO:tensorflow:step = 401, loss = 2.01063 (0.249 sec)
INFO:tensorflow:Saving checkpoints for 500 into /var/folders/y0/f6x1qz191gxdq8fj4lpy82l00000gn/T/tmpjj9r6j1j/model.ckpt.
INFO:tensorflow:Loss for final step: 0.49854.


In [29]:
# Create an input function for evaluation
input_fnc_eval = tf.estimator.inputs.pandas_input_fn(X_test, shuffle=False)

In [30]:
# Get predictions and cast it into a list
preds = list(model.predict(input_fn=input_fnc_eval))

INFO:tensorflow:Restoring parameters from /var/folders/y0/f6x1qz191gxdq8fj4lpy82l00000gn/T/tmpjj9r6j1j/model.ckpt-500


In [31]:
# Select only the predicted class ids
predictions = [p['class_ids'][0] for p in preds]

In [32]:
# Evaluate the model
from sklearn.metrics import classification_report

In [33]:
print(classification_report(y_test, predictions))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        13
          1       1.00      0.55      0.71        20
          2       0.57      1.00      0.73        12

avg / total       0.89      0.80      0.80        45



In [34]:
# Create a linear model
linear_model = tf.estimator.LinearClassifier(feature_columns=feat_cols, n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_master': '', '_log_step_count_steps': 100, '_keep_checkpoint_max': 5, '_task_id': 0, '_task_type': 'worker', '_session_config': None, '_service': None, '_keep_checkpoint_every_n_hours': 10000, '_num_worker_replicas': 1, '_tf_random_seed': None, '_model_dir': '/var/folders/y0/f6x1qz191gxdq8fj4lpy82l00000gn/T/tmpmvnx9oqb', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1a13d4a128>, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_save_summary_steps': 100, '_is_chief': True}


In [35]:
# Train the model
linear_model.train(input_fn=input_fnc)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/y0/f6x1qz191gxdq8fj4lpy82l00000gn/T/tmpmvnx9oqb/model.ckpt.
INFO:tensorflow:step = 1, loss = 10.9861
INFO:tensorflow:global_step/sec: 408.305
INFO:tensorflow:step = 101, loss = 4.97176 (0.247 sec)
INFO:tensorflow:global_step/sec: 441.932
INFO:tensorflow:step = 201, loss = 2.28684 (0.226 sec)
INFO:tensorflow:global_step/sec: 424.618
INFO:tensorflow:step = 301, loss = 2.46504 (0.235 sec)
INFO:tensorflow:global_step/sec: 427.412
INFO:tensorflow:step = 401, loss = 2.28224 (0.234 sec)
INFO:tensorflow:global_step/sec: 657.873
INFO:tensorflow:step = 501, loss = 2.04661 (0.152 sec)
INFO:tensorflow:Saving checkpoints for 525 into /var/folders/y0/f6x1qz191gxdq8fj4lpy82l00000gn/T/tmpmvnx9oqb/model.ckpt.
INFO:tensorflow:Loss for final step: 1.08493.


In [36]:
# Get predictions and cast it into a list
preds = list(linear_model.predict(input_fn=input_fnc_eval))

INFO:tensorflow:Restoring parameters from /var/folders/y0/f6x1qz191gxdq8fj4lpy82l00000gn/T/tmpmvnx9oqb/model.ckpt-525


In [37]:
# Select only the class ids
predictions = [p['class_ids'][0] for p in preds]

In [38]:
# Evaluate the model
print(classification_report(y_test, predictions))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        13
          1       0.95      1.00      0.98        20
          2       1.00      0.92      0.96        12

avg / total       0.98      0.98      0.98        45

